In [1]:
import empyrical as ep
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import pyfolio as pf
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import yfinance as yf

import datetime
import warnings
import pytz

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

plt.style.use("tableau-colorblind10")
%matplotlib inline

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
# Disable future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [3]:
tickers = [
    "AAV", "ADVANC", "AEONTS", "AMATA", "AOT", "AP", "AURA", "AWC", "BAM", "BANPU",
    "BBL", "BCH", "BCP", "BCPG", "BDMS", "BEM", "BGRIM", "BH", "BLA", 
    # "BSRC",
    "BTG", "BTS", "BYD", "CBG", "CENTEL", "CHG", "CK", "COM7", "CPALL", "CPF",
    "CPN", "CRC", "DELTA", "DOHOME", "EA", "EGCO", "ERW", "FORTH", "GLOBAL", "GPSC",
    "GULF", "GUNKUL", "HANA", "HMPRO", "ICHI", "INTUCH", "IRPC", "ITC", "IVL", "JMART",
    "JMT", "KBANK", "KCE", "KKP", "KTB", "KTC", "LH", "M", "MEGA", "MINT",
    "MOSHI", "MTC", "NEX", "OR", "ORI", "OSP", "PLANB", "PTT", "PTTEP", "PTTGC",
    "RATCH", "RBF", "RCL", "SAPPE", "SAWAD", "SCB", "SCC", "SCGP", "SIRI", "SISB",
    "SJWD", "SNNP", "SPALI", "SPRC", "STA", "STGT", "TASCO", "TCAP", "THG", "TIDLOR",
    "TISCO", "TKN", "TLI", "TOA", "TOP", "TRUE", "TTB", "TU", "VGI", "WHA"
]

In [4]:
tickers

['AAV',
 'ADVANC',
 'AEONTS',
 'AMATA',
 'AOT',
 'AP',
 'AURA',
 'AWC',
 'BAM',
 'BANPU',
 'BBL',
 'BCH',
 'BCP',
 'BCPG',
 'BDMS',
 'BEM',
 'BGRIM',
 'BH',
 'BLA',
 'BTG',
 'BTS',
 'BYD',
 'CBG',
 'CENTEL',
 'CHG',
 'CK',
 'COM7',
 'CPALL',
 'CPF',
 'CPN',
 'CRC',
 'DELTA',
 'DOHOME',
 'EA',
 'EGCO',
 'ERW',
 'FORTH',
 'GLOBAL',
 'GPSC',
 'GULF',
 'GUNKUL',
 'HANA',
 'HMPRO',
 'ICHI',
 'INTUCH',
 'IRPC',
 'ITC',
 'IVL',
 'JMART',
 'JMT',
 'KBANK',
 'KCE',
 'KKP',
 'KTB',
 'KTC',
 'LH',
 'M',
 'MEGA',
 'MINT',
 'MOSHI',
 'MTC',
 'NEX',
 'OR',
 'ORI',
 'OSP',
 'PLANB',
 'PTT',
 'PTTEP',
 'PTTGC',
 'RATCH',
 'RBF',
 'RCL',
 'SAPPE',
 'SAWAD',
 'SCB',
 'SCC',
 'SCGP',
 'SIRI',
 'SISB',
 'SJWD',
 'SNNP',
 'SPALI',
 'SPRC',
 'STA',
 'STGT',
 'TASCO',
 'TCAP',
 'THG',
 'TIDLOR',
 'TISCO',
 'TKN',
 'TLI',
 'TOA',
 'TOP',
 'TRUE',
 'TTB',
 'TU',
 'VGI',
 'WHA']

In [5]:
# Function to fetch data for a ticker
def fetch_data(ticker, start_date, end_date):
    dft = yf.Ticker(ticker)
    df = dft.history(interval="1d", start=start_date, end=end_date)
    df['Ticker'] = ticker
    return df

In [6]:
tickers_list = [ ticker + ".BK" for ticker in tickers ]
tickers_list.insert(0,"TDEX.BK")
tickers_list.insert(0,"^SET.BK")

In [7]:
news = pd.read_csv("counted_news.csv")

In [8]:
news

,Unnamed: 0,Date,stockList,StockSet,totalCount,maxMentioned,maxCount,3DaySet
0,0,01/01/2024,[],NaN,NaN,NaN,NaN,[]
1,1,02/01/2024,"['AAI', 'AAI', 'AI', 'AI', 'AOT', 'AOT', 'BBL'...","['AAI', 'AI', 'AOT', 'BBL', 'BIG', 'DIF', 'EA'...",40.0,NEW,5.0,"['AAI', 'AI', 'AOT', 'BBL', 'BIG', 'DIF', 'EA'..."
2,2,03/01/2024,"['AAV', 'ADVANC', 'ADVANC', 'ADVANC', 'AI', 'A...","['AAV', 'ADVANC', 'AI', 'ALL', 'AOT', 'AQUA', ...",202.0,KEX,15.0,"['AAI', 'AAV', 'ADVANC', 'AI', 'ALL', 'AOT', '..."
3,3,04/01/2024,"['AAV', 'AAV', 'AAV', 'AEONTS', 'AGE', 'AH', '...","['AAV', 'AEONTS', 'AGE', 'AH', 'AI', 'AIRA', '...",249.0,AOT,9.0,"['AAI', 'AAV', 'ADVANC', 'AEONTS', 'AGE', 'AH'..."
4,4,05/01/2024,"['A5', 'AAV', 'AAV', 'ACG', 'ADVANC', 'AH', 'A...","['A5', 'AAV', 'ACG', 'ADVANC', 'AH', 'AI', 'AI...",291.0,SPRC,17.0,"['A5', 'AAV', 'ACG', 'ADVANC', 'AEONTS', 'AGE'..."
...,...,...,...,...,...,...,...,...
180,180,29/06/2024,"['AAI', 'AAI', 'ADB', 'ADB', 'ADVANC', 'AHC', ...","['AAI', 'ADB', 'ADVANC', 'AHC', 'AI', 'AIT', '...",285.0,EA,49.0,"['24CS', 'A5', 'AAI', 'ACE', 'ADB', 'ADD', 'AD..."
181,181,30/06/2024,"['AI', 'AI', 'AI', 'AI', 'AI', 'AOT', 'AP', 'B...","['AI', 'AOT', 'AP', 'BBGI', 'BDMS', 'BEM', 'BG...",70.0,HEALTH,6.0,"['24CS', 'A5', 'AAI', 'ADB', 'ADD', 'ADVANC', ..."
182,182,01/07/2024,"['ADD', 'ADVANC', 'AI', 'AI', 'AI', 'AI', 'AI'...","['ADD', 'ADVANC', 'AI', 'ALL', 'BGRIM', 'BTC',...",57.0,AI,5.0,"['AAI', 'ADB', 'ADD', 'ADVANC', 'AHC', 'AI', '..."
183,183,02/07/2024,"['ADD', 'ADD', 'ADVANC', 'ADVANC', 'ADVANC', '...","['ADD', 'ADVANC', 'ADVICE', 'AEONTS', 'AI', 'A...",310.0,EA,31.0,"['ADD', 'ADVANC', 'ADVICE', 'AEONTS', 'AI', 'A..."


In [9]:
# Define the start and end dates for the data
startDate = "2015-06-20"
endDate = "2024-06-20"

# Create an empty DataFrame to store all data
all_data = pd.DataFrame()

# tickers_list = ["^SET.BK", "AWC.BK"]

showModelScores = False

gridSearch = False

cvTest = False

df = pd.DataFrame()

# Loop through each ticker
for ticker in tickers_list:

    df = fetch_data(ticker, startDate, endDate)
    dft = yf.Ticker(ticker)
    timeZone = dft.info.get("timeZoneFullName")
    tickerName = dft.info.get("longName", "Unknown Ticker")

    # Drop columns
    df.drop(columns=['Dividends'], inplace=True)
    # Drop Stock Splits column
    df.drop(columns=['Stock Splits'], inplace=True)

    ## Calculate EMA-12 and EMA-26 using Exponential Weighing Average (EWM)
    # df['EMA-12'] = df['Close'].ewm(span = 12, adjust = False).mean()
    # df['EMA-26'] = df['Close'].ewm(span = 26, adjust = False).mean()

    ## Calculate MACD
    # df['MACD'] = df['EMA-12'] - df['EMA-26']
    df['MACD'] = ta.macd(df['Close'], fast=12, slow=26, signal=9)['MACD_12_26_9']
    df['Pct_Change'] = df['Close'].pct_change() * 100

    ## Calculate RSI using formula
    ## RSI = 100 – [100 ÷ ( 1 + (Average Gain During Up Periods ÷ Average Loss During Down Periods ))]

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the Exponential Moving Average of gains and losses
    avg_gain = gain.ewm(span=14, min_periods=14).mean()
    avg_loss = loss.ewm(span=14, min_periods=14).mean()

    # Calculate the RS and RSI
    rs = avg_gain / avg_loss
    df['RSI_EMA'] = 100 - (100 / (1 + rs))

    df['RSI_ta'] = ta.rsi(df['Close'], length=14)

    df['MA10'] = df.ta.sma(length=10)
    df['MA50'] = df.ta.sma(length=50)
    df['MA200'] = df.ta.sma(length=200)

    ## Calculate Boilinger Bands
    window = 20
    df['MA20'] = df['Close'].rolling(window=window).mean()
    df['std_dev'] = df['Close'].rolling(window=window).std()
    df['Upper_BB'] = df['MA20'] + (df['std_dev'] * 2)
    df['Lower_BB'] = df['MA20'] - (df['std_dev'] * 2)

    # Create target variable: 1 if next day's close is higher than today's, else 0
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

    baro = pd.read_csv('sentiment_score.csv')
    baro['Close'] = baro['Baro_Index']
    baro['BARO-EMA2'] = baro.ta.ema(length=2)
    baro['BARO-EMA3'] = baro.ta.ema(length=3)
    baro['BARO-EMA10'] = baro.ta.ema(length=10)
    baro['BARO-EMA50'] = baro.ta.ema(length=50)
    baro['BARO-EMA200'] = baro.ta.ema(length=200)
    
    baro['BARO-SMA2'] = baro.ta.sma(length=2)
    baro['Date'] = pd.to_datetime(baro['CreateDate'], format='%d/%m/%Y')
    # Sort the DataFrame by CreateDate
    baro_sorted = baro.sort_values(by='Date')
    # Reset the index if needed
    baro_sorted = baro_sorted.reset_index(drop=True)
    baro_sorted.set_index('Date', inplace=True)
    baro_sorted.index = baro_sorted.index.tz_localize('UTC').tz_convert('Asia/Bangkok')
    baro_sorted.index = baro_sorted.index.normalize()

    df['Baro'] = baro_sorted['Close']
    df['Baro-2'] = baro_sorted['BARO-EMA2']
    df['Baro-S2'] = baro_sorted['BARO-SMA2']
    # df['Baro-10'] = baro_sorted['BARO-EMA10']
    # df['Baro-50'] = baro_sorted['BARO-EMA50']
    # df['Baro-200'] = baro_sorted['BARO-EMA200']

    df = df.dropna()

    # Prepare the feature set and target variable
    X = df[['Pct_Change', 'std_dev', 'MA200', 'MA50', 'MA10', 'MACD', 'RSI_ta', 'Baro']]
    # X = df[['MA200', 'MA50', 'MA10', 'MACD2', 'RSI_ta', 'VIX', 'VIX_35', 'VIX_65']]
    # X = df[['SMA200', 'SMA50', 'SMA10', 'MACD', 'RSI', 'VIX', 'VIX_35', 'VIX_65', 'MA20', 'Lower_BB', 'Upper_BB']]
    y = df['Target']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    rf = RandomForestClassifier (
        random_state=42, 
        max_depth=5, 
        min_samples_leaf=1, 
        min_samples_split=5, 
        n_estimators=91
    )

    xgb = XGBClassifier (
        random_state=42, 
        colsample_bytree=0.8, 
        gamma=0.1, 
        learning_rate=0.001, 
        max_depth=3, 
        n_estimators=100, 
        reg_alpha=0.1, 
        reg_lambda=0.1, 
        subsample=0.8
    )

    model = VotingClassifier(estimators=[('rf', rf), ('xgb', xgb)], voting='soft')
    model.fit(X_train, y_train)

    if showModelScores:
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"Accuracy of the model: {acc:.4f}")

        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.show()

    BaroSD = 13.14880598
    BaroMean = 54.67908753
    BaroCount = 1885

    Baro_MINUS_1_SD = BaroMean - ( 1 * BaroSD )
    Baro_MINUS_15_SD = BaroMean - ( 1.5 * BaroSD )
    Baro_MINUS_2_SD = BaroMean - ( 2 * BaroSD )

    Baro_PLUS_1_SD = BaroMean + ( 1 * BaroSD )
    Baro_PLUS_15_SD = BaroMean + ( 1.5 * BaroSD )
    Baro_PLUS_2_SD = BaroMean + ( 2 * BaroSD )

    # print(f"{Baro_MINUS_1_SD:.3f} | {Baro_PLUS_1_SD:.3f}")
    # print(f"{Baro_MINUS_15_SD:.3f} | {Baro_PLUS_15_SD:.3f}")
    # print(f"{Baro_MINUS_2_SD:.3f} | {Baro_PLUS_2_SD:.3f}")

    df['Signal'] = 0
    # df.loc[X_test.index, 'Signal'] = model.predict(X_test) ## Model 1 Random Forest
    # df.loc[X_test.index, 'Signal'] = ensemble_model1.predict(X_test) ## Model 2 Random Forest + XGBoost
    df.loc[X_test.index, 'Signal'] = model.predict(X_test) ## Model 3 More precise Random Forest + XGBoost 

    # Initialize the 'Position' column
    df['Position'] = 0

    holding_position = False

    order_list = []

    start_idx = df.index.get_loc(X_test.index[0])

    ol = []
    cl = []

    date_list = news["Date"].tolist()

    for i in range(start_idx, len(df)):

        curDate = str(df.index[i].date())

        stockList = []

        curData = news[news["Date"] == curDate]

        if curDate in date_list:
            stockList = curData["stockList"].iloc[0]

        if ticker.replace("^", "")[:-3] not in stockList:
            continue
        elif df.loc[df.index[i], 'Baro-S2'] < Baro_MINUS_1_SD and not holding_position:
            df.loc[df.index[i], 'Position'] = 1
            holding_position = True
            ol.append(df.index[i])
        elif df.loc[df.index[i], 'Baro-S2'] > Baro_PLUS_1_SD and holding_position:
            df.loc[df.index[i], 'Position'] = 0
            holding_position = False
            cl.append(df.index[i])

    df.loc[df['Position'] == 0, 'Position'] = np.nan
    df['Position'] = df['Position'].fillna(method='ffill')
    df['Position'] = df['Position'].fillna(0).astype(int)

    df_test_period = df.loc[X_test.index[0]:]
    df_test_period = df_test_period.copy()
    df_test_period.loc[:, 'Strategy_Returns'] = df_test_period['Position'].shift(1) * df_test_period['Close'].pct_change()
    df_test_period.dropna(subset=['Strategy_Returns'], inplace=True)

    returns = df_test_period['Strategy_Returns']

    cumulative_returns = (1 + returns).cumprod() - 1
    perf_stats = {
        'Annual Return': ep.annual_return(returns),
        'Cumulative Returns': ep.cum_returns_final(returns),
        'Annual Volatility': ep.annual_volatility(returns),
        'Sharpe Ratio': ep.sharpe_ratio(returns),
        'Sortino Ratio': ep.sortino_ratio(returns),
        'Max Drawdown': ep.max_drawdown(returns),
        'Calmar Ratio': ep.calmar_ratio(returns)
    }

    perf_df = pd.DataFrame(perf_stats, index=[ticker])

    perf_df['Index Name'] = tickerName

    perf_df['1_Train'] = model.score(X_train,y_train)
    perf_df['1_Test'] = model.score(X_test,y_test)

    perf_df['NumOfOrders'] = len(ol)

    perf_df['Test Date'] = str(X_test.index[0])

    perf_df = perf_df[['Index Name'] + [col for col in perf_df.columns if col != 'Index Name']]

    # Append to all_data DataFrame
    all_data = pd.concat([all_data, perf_df])

    print(f"{ticker} | Train : {model.score(X_train, y_train):.4f} | Test : {model.score(X_test, y_test):.4f}")
    print(f"{ep.annual_return(returns):.5f} {ep.annual_volatility(returns):.5f} {ep.sharpe_ratio(returns):.5f} {ep.max_drawdown(returns):.5f}")

^SET.BK | Train : 0.6540 | Test : 0.4800
0.00000 0.00000 nan 0.00000
TDEX.BK | Train : 0.6953 | Test : 0.5851
0.00000 0.00000 nan 0.00000
AAV.BK | Train : 0.6667 | Test : 0.6223
0.00000 0.00000 nan 0.00000
ADVANC.BK | Train : 0.5895 | Test : 0.6117
0.00000 0.00000 nan 0.00000
AEONTS.BK | Train : 0.6021 | Test : 0.6144
0.00000 0.00000 nan 0.00000
AMATA.BK | Train : 0.5975 | Test : 0.6037
0.00000 0.00000 nan 0.00000
AOT.BK | Train : 0.6048 | Test : 0.6330
0.00000 0.00000 nan 0.00000
AP.BK | Train : 0.6041 | Test : 0.6410
0.00000 0.00000 nan 0.00000
AURA.BK | Train : 0.7538 | Test : 0.6364
0.00000 0.00000 nan 0.00000
AWC.BK | Train : 0.6260 | Test : 0.6393
0.00000 0.00000 nan 0.00000
BAM.BK | Train : 0.6466 | Test : 0.6629
0.00000 0.00000 nan 0.00000
BANPU.BK | Train : 0.6267 | Test : 0.6649
0.00000 0.00000 nan 0.00000
BBL.BK | Train : 0.6048 | Test : 0.6064
0.00000 0.00000 nan 0.00000
BCH.BK | Train : 0.6201 | Test : 0.5984
0.00000 0.00000 nan 0.00000
BCP.BK | Train : 0.6121 | Test : 0.5

In [10]:
df

,Open,High,Low,Close,Volume,Ticker,MACD,Pct_Change,RSI,RSI_EMA,...,MA20,std_dev,Upper_BB,Lower_BB,Target,Baro,Baro-2,Baro-S2,Signal,Position
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-22 00:00:00+07:00,2.548177,2.744191,2.518021,2.698957,397621700,WHA.BK,0.053531,5.917141,74.999982,80.485509,...,2.406444,0.093767,2.593979,2.218910,1,9.42,26.115000,26.115,0,0
2016-08-23 00:00:00+07:00,2.698957,2.744191,2.683879,2.744191,168519300,WHA.BK,0.071504,1.675980,78.260892,82.481741,...,2.424538,0.120087,2.664712,2.184364,0,13.72,17.851667,11.570,0,0
2016-08-24 00:00:00+07:00,2.744191,2.744191,2.653723,2.683879,193534100,WHA.BK,0.079959,-2.197808,72.000004,71.266100,...,2.440370,0.132379,2.705127,2.175612,1,3.03,7.970556,8.375,0,0
2016-08-25 00:00:00+07:00,2.698956,2.789424,2.668801,2.729112,335470800,WHA.BK,0.089281,1.685386,73.584903,74.291291,...,2.456955,0.146715,2.750386,2.163525,0,4.26,5.496852,3.645,0,0
2016-08-26 00:00:00+07:00,2.729112,2.789424,2.714034,2.729112,193324600,WHA.BK,0.095567,0.000000,74.999985,74.291291,...,2.473541,0.157948,2.789437,2.157645,0,7.35,6.732284,5.805,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13 00:00:00+07:00,5.300000,5.400000,5.250000,5.250000,44198000,WHA.BK,0.078698,-0.943400,40.000000,39.019974,...,5.390000,0.096791,5.583581,5.196419,1,62.92,58.423356,57.395,0,0
2024-06-14 00:00:00+07:00,5.250000,5.400000,5.250000,5.300000,31658900,WHA.BK,0.068671,0.952385,42.857156,45.169406,...,5.392500,0.093577,5.579653,5.205347,0,47.49,51.134452,55.205,0,0
2024-06-17 00:00:00+07:00,5.300000,5.300000,5.300000,5.300000,0,WHA.BK,0.060033,0.000000,40.000000,45.169406,...,5.395000,0.090175,5.575350,5.214650,0,58.47,56.024817,52.980,0,0


In [11]:
df.head()

,Open,High,Low,Close,Volume,Ticker,MACD,Pct_Change,RSI,RSI_EMA,...,MA20,std_dev,Upper_BB,Lower_BB,Target,Baro,Baro-2,Baro-S2,Signal,Position
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-22 00:00:00+07:00,2.548177,2.744191,2.518021,2.698957,397621700,WHA.BK,0.053531,5.917141,74.999982,80.485509,...,2.406444,0.093767,2.593979,2.218910,1,9.42,26.115000,26.115,0,0
2016-08-23 00:00:00+07:00,2.698957,2.744191,2.683879,2.744191,168519300,WHA.BK,0.071504,1.675980,78.260892,82.481741,...,2.424538,0.120087,2.664712,2.184364,0,13.72,17.851667,11.570,0,0
2016-08-24 00:00:00+07:00,2.744191,2.744191,2.653723,2.683879,193534100,WHA.BK,0.079959,-2.197808,72.000004,71.266100,...,2.440370,0.132379,2.705127,2.175612,1,3.03,7.970556,8.375,0,0
2016-08-25 00:00:00+07:00,2.698956,2.789424,2.668801,2.729112,335470800,WHA.BK,0.089281,1.685386,73.584903,74.291291,...,2.456955,0.146715,2.750386,2.163525,0,4.26,5.496852,3.645,0,0
2016-08-26 00:00:00+07:00,2.729112,2.789424,2.714034,2.729112,193324600,WHA.BK,0.095567,0.000000,74.999985,74.291291,...,2.473541,0.157948,2.789437,2.157645,0,7.35,6.732284,5.805,0,0


In [12]:
all_data.dropna(inplace=True)

In [13]:
all_data

,Index Name,Annual Return,Cumulative Returns,Annual Volatility,Sharpe Ratio,Sortino Ratio,Max Drawdown,Calmar Ratio,1_Train,1_Test,NumOfOrders,Test Date


In [14]:
# all_data.to_csv("set50_return.csv")
all_data.to_excel("BaroSET100wh.xlsx", index=True)

PermissionError: [Errno 13] Permission denied: 'BaroSET100wh.xlsx'

In [ ]:
df2 = all_data

In [ ]:
Q1 = df2['Annual Return'].quantile(0.25)
Q3 = df2['Annual Return'].quantile(0.75)

# Compute the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define lower and upper bound
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df2

# Filter the DataFrame to remove outliers
df_filtered = df2[(df2['Annual Return'] >= lower_bound) & (df2['Annual Return'] <= upper_bound)]

In [ ]:
# df_filtered.to_excel("FilteredSET100wh.xlsx", index=True)